# Experiment 5.0 - Inception Resnet v2 - extract Features

Reproduce Results of [Transfer learning with deep convolutional neural network for liver steatosis assessment in ultrasound images](https://pubmed.ncbi.nlm.nih.gov/30094778/). We used a pre-trained CNN to extract features based on B-mode images. 

The CNNfeatures are extracted using the pretrained Inception-Resnet-v2 implemented in Keras.
See reference: https://jkjung-avt.github.io/keras-inceptionresnetv2/

![Screen Shot 2020-10-12 at 2 57 22 PM](https://user-images.githubusercontent.com/23482039/95781182-47437700-0c9b-11eb-8826-594811ba3322.png)


In [1]:
import sys
import random
sys.path.append('../src')

import warnings
warnings.filterwarnings("ignore") 

import pickle
import pandas as pd

import numpy as np
import mlflow
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from tqdm import tqdm

from utils.compute_metrics import get_metrics, get_majority_vote,log_test_metrics
from utils.dataframe_creation import create_dataframe_preproccessing
from sklearn.preprocessing import StandardScaler
from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from pprint import pprint
from itertools import product
from keras.preprocessing.image import ImageDataGenerator


## 1. Feature Extraction

In [2]:
M, N= 434, 636 # ultrasound image dimension
#In the paper, the authors extract the features directly from the pretrained inception resnet_v2
# pooling: Optional pooling mode for feature extraction when include_top is False.
#'max' means that global max pooling will be applied.
net = InceptionResNetV2(include_top=False,
                        weights='imagenet',
                        pooling= 'avg')
dataset = create_dataframe_preproccessing()
dataset.head()

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,id,labels,fat,fname
0,1,0,3,../data/02_interim/raw_images/1/P1_image1.jpg
1,1,0,3,../data/02_interim/raw_images/1/P1_image2.jpg
2,1,0,3,../data/02_interim/raw_images/1/P1_image3.jpg
3,1,0,3,../data/02_interim/raw_images/1/P1_image4.jpg
4,1,0,3,../data/02_interim/raw_images/1/P1_image5.jpg


In [3]:
dataset = dataset.astype({"labels": str})
datagen = ImageDataGenerator(preprocessing_function= tf.keras.applications.inception_resnet_v2.preprocess_input)
#datagen = ImageDataGenerator(rescale=1./255)
#datagen = ImageDataGenerator(featurewise_center=True,
                                   #featurewise_std_normalization=True)
#datagen.mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
#datagen.std = np.array([0.229, 0.224, 0.225], dtype=np.float32)

#From paper: Images were resized using the bi-cubic interpolation algorithm to the resolution originally designed for the network.
generator = datagen.flow_from_dataframe(dataset, x_col='fname',y_col='labels',shuffle=False, target_size=(299,299), interpolation ='bicubic', batch_size =25)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Found 550 validated image filenames belonging to 2 classes.


In [4]:
max_pool_dim = 1536
features = np.zeros(shape=(len(dataset), max_pool_dim))
batch_size = 25
assert len(dataset)%batch_size ==0, '550 should be divisable by batch_SIZE'

for i, (inputs_batch, labels_batch) in enumerate(tqdm(generator)):
    features[i*batch_size:(i+1)*batch_size] = net(inputs_batch)
    if (i+1)*batch_size == 550:
        break

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 95%|█████████▌| 21/22 [01:09<00:03,  3.29s/it]


## 2. Save features

In [5]:
df_features = pd.DataFrame(features)
df_features= pd.concat([dataset['id'], dataset['labels'], df_features], axis=1)
df_features[["labels"]] = df_features[["labels"]].apply(pd.to_numeric)
df_features.head()

,id,labels,0,1,2,3,4,5,6,7,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,1,0,0.646588,0.537462,0.401942,0.199507,0.202658,0.294940,0.500031,0.821503,...,0.229984,0.633566,0.206114,0.732518,0.654043,0.275888,0.362820,0.107184,0.501511,0.489348
1,1,0,0.693356,0.374550,0.360310,0.129162,0.239083,0.262955,0.478658,0.791732,...,0.192122,0.485892,0.299204,0.655221,0.580703,0.341523,0.230004,0.092797,0.491484,0.658840
2,1,0,0.693498,0.392268,0.309536,0.125567,0.170290,0.238435,0.447406,0.975265,...,0.134828,0.366664,0.171025,0.465990,0.529727,0.310153,0.244650,0.107581,0.403802,0.578181
3,1,0,0.612398,0.441856,0.305321,0.126919,0.219824,0.259561,0.441000,0.850824,...,0.268535,0.460096,0.280195,0.543407,0.608157,0.337704,0.300175,0.188300,0.455578,0.640826
4,1,0,0.555925,0.398252,0.232699,0.125582,0.220398,0.237641,0.377193,0.840379,...,0.196321,0.381724,0.252318,0.516770,0.624552,0.336688,0.322390,0.077366,0.416018,0.598994


In [6]:
# Create a dict for the scattering features, and parameters
inception_dict_tensor = {
                'features':df_features,
                'label' : dataset['labels'],
                'pid' : dataset['id'],
                'Interpolation': 'bicubic'}

with open('../data/03_features/inception_dict_tensor_avg_interpolation_pooling.pickle', 'wb') as handle:
    pickle.dump(inception_dict_tensor, handle, protocol=pickle.HIGHEST_PROTOCOL)

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
